In [1]:
from mylib import preprocess
import pandas as pd
import os
import numpy as np

pretest = pd.read_csv(os.path.join('data', 'test_users.csv'), header=0, parse_dates=[1,2,3])
pretrain = pd.read_csv(os.path.join('data', 'train_users_2.csv'), header=0, parse_dates=[1,2,3])
df_sessions = pd.read_csv("data/sessions.csv", encoding='utf8')

In [2]:
train, test, y, le = preprocess.make_user_features(pretrain, pretest)

In [3]:
data = pd.concat((train, test), axis=0, ignore_index=True)
final = preprocess.make_sessions_features(data, df_sessions)

del data

In [4]:
X_train = final.ix[:train.shape[0]-1]
X_test = final.ix[train.shape[0]:]

In [ ]:
assert train.shape[0] == y.shape[0]

In [8]:
assert X_train.shape[0] == train.shape[0]
assert X_train.shape[0] == y.shape[0]
print X_train.shape
print X_test.shape
print y.shape

(213451, 549)
(62096, 549)
(213451,)


In [9]:
del final

In [10]:
from sklearn.ensemble import BaggingClassifier
def bagging_prediction(X_train, y_train, X_test, 
                       n_estimators=100, 
                       max_samples=0.1, 
                       max_features=1.0, 
                       random_state=None):

    unimportant_features = np.load("unimportant_features.npy")
    bagg = BaggingClassifier(random_state=random_state, 
                             n_estimators=n_estimators, 
                             max_samples=max_samples, 
                             max_features=max_features)
    bagg.fit(X_train.ix[:, ~unimportant_features], y_train)
    return bagg.predict_proba(X_test.ix[:, ~unimportant_features])

In [ ]:
probs = []
for i in range(5):
    p = bagging_prediction(X_train, y, 
                           X_test,
                           n_estimators=100,
                           random_state=i)
    probs.append(p)

# We take the average
avg_probs = sum(probs)/len(probs)

In [ ]:
y_pred = avg_probs
ids = []  #list of ids
cts = []  #list of countries
id_test = pretest.id.values
for i in range(len(id_test)):
    idx = id_test[i]
    ids += [idx] * 5
    cts += le.inverse_transform(np.argsort(y_pred[i])[::-1])[:5].tolist()

#Generate submission
sub = pd.DataFrame(np.column_stack((ids, cts)), columns=['id', 'country'])
sub.to_csv(os.path.join('data', 'sub9.csv'),index=False)